## Uniswap V3 Architecture (Math Explanation)

In [2]:
import numpy as np
import math

 we’re going to buy ETH for USDC at the price of $5000 per 1 ETH.
 
 We’ll need to find three ticks:

    The current tick will correspond to the current price (5000 USDC for 1 ETH).
    The lower and upper bounds of the price range we’re providing liquidity into. Let the lower price be $4545 and the upper price be $5500.

In [ ]:
# Calculating price
def calculating_price(x:float, y:float) -> float:
    return math.sqrt(y, x)

# Calculating price tick
def price_to_tick(p:float) -> float:
    p = math.sqrt(p)
    return math.floor(math.log(p, 1.0001))

q96 = 2**96
def price_to_sqrtp(p:float) -> int:
    return int(math.sqrt(p) * q96)


# Liquidity Amount Calculation
sqrtp_low = price_to_sqrtp(4545)
sqrtp_cur = price_to_sqrtp(5000)
sqrtp_upp = price_to_sqrtp(5500)

def liquidity0(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

eth = 10**18
amount_eth = 1 * eth
amount_usdc = 5000 * eth

liq0 = liquidity0(amount_eth, sqrtp_cur, sqrtp_upp)
liq1 = liquidity1(amount_usdc, sqrtp_cur, sqrtp_low)
liq = int(min(liq0, liq1))
# > 1517882343751509868544


# Token Amounts Calculation
def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)

def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

amount0 = calc_amount0(liq, sqrtp_upp, sqrtp_cur)
amount1 = calc_amount1(liq, sqrtp_low, sqrtp_cur)
(amount0, amount1)
# > (998976618347425408, 5000000000000000000000)

